<a href="https://colab.research.google.com/github/kiki4510/Spam-E-mail-Classification/blob/main/Section4_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#필요한 라이브러리 import
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import random
random_state = 42

random.seed(random_state)
np.random.seed(random_state)


In [ ]:
#구글드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/MyDrive

/content/drive/MyDrive


In [ ]:
df = pd.read_csv('Spam Email raw text for NLP.csv')
df.head()

,CATEGORY,MESSAGE,FILE_NAME
0,1,"Dear Homeowner,\n\n \n\nInterest Rates are at ...",00249.5f45607c1bffe89f60ba1ec9f878039a
1,1,ATTENTION: This is a MUST for ALL Computer Use...,00373.ebe8670ac56b04125c25100a36ab0510
2,1,This is a multi-part message in MIME format.\n...,00214.1367039e50dc6b7adb0f2aa8aba83216
3,1,IMPORTANT INFORMATION:\n\n\n\nThe new domain n...,00210.050ffd105bd4e006771ee63cabc59978
4,1,This is the bottom line. If you can GIVE AWAY...,00033.9babb58d9298daa2963d4f514193d7d6


In [ ]:
#텍스트 전처리 작업
df['MESSAGE'] = df['MESSAGE'].apply(lambda x: x.lower())


In [ ]:
import keras
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

In [ ]:
#토큰화 함수
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
def tokenizer_sequences(num_words, X):
    
    tokenizer = Tokenizer(num_words=num_words)
    tokenizer.fit_on_texts(X)
    sequences = tokenizer.texts_to_sequences(X)
    
    return tokenizer, sequences

In [ ]:
max_words = 10000 
max_len = 300
tokenizer, sequences = tokenizer_sequences(max_words, df.MESSAGE.copy())

In [ ]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

X = pad_sequences(sequences, maxlen=max_len)
y = df.CATEGORY.copy()

Found 104210 unique tokens.


In [ ]:
max_words = len(tokenizer.word_index) + 1
embedding_dim = 100

In [ ]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=max_len))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 100)          10421100  
                                                                 
 flatten (Flatten)           (None, 30000)             0         
                                                                 
 dense (Dense)               (None, 32)                960032    
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 11,381,165
Trainable params: 11,381,165
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#훈련과 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['acc'])
record = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
116/116 [==============================] - 4s 17ms/step - loss: 0.2115 - acc: 0.9137 - val_loss: 0.0458 - val_acc: 0.9903
Epoch 2/10
116/116 [==============================] - 2s 15ms/step - loss: 0.0127 - acc: 0.9965 - val_loss: 0.0175 - val_acc: 0.9968
Epoch 3/10
116/116 [==============================] - 2s 15ms/step - loss: 0.0040 - acc: 0.9987 - val_loss: 0.0230 - val_acc: 0.9914
Epoch 4/10
116/116 [==============================] - 2s 15ms/step - loss: 0.0038 - acc: 0.9987 - val_loss: 0.0408 - val_acc: 0.9828
Epoch 5/10
116/116 [==============================] - 2s 15ms/step - loss: 0.0020 - acc: 0.9995 - val_loss: 0.0257 - val_acc: 0.9881
Epoch 6/10
116/116 [==============================] - 2s 15ms/step - loss: 0.0017 - acc: 0.9992 - val_loss: 0.0220 - val_acc: 0.9935
Epoch 7/10
116/116 [==============================] - 2s 14ms/step - loss: 7.8968e-04 - acc: 0.9997 - val_loss: 0.0201 - val_acc: 0.9946
Epoch 8/10
116/116 [==============================] - 2s 15ms/ste

In [ ]:
model.evaluate(X_test, y_test)

37/37 [==============================] - 0s 5ms/step - loss: 0.0262 - acc: 0.9922


[0.02619043178856373, 0.992241382598877]

In [ ]:
#chance level 확인
df['CATEGORY'].value_counts()

0    3900
1    1896
Name: CATEGORY, dtype: int64

In [ ]:
#교차 검증을 통한 일반화 가능성 확인
from sklearn.model_selection import KFold, StratifiedKFold
kf = KFold(n_splits = 5)
skf = StratifiedKFold(n_splits = 5, random_state = 42, shuffle = True) 
x_train = pd.DataFrame(X_train)
y_train = pd.DataFrame(y_train)

for train_index, val_index in kf.split(np.zeros(x_train.shape[0]),y_train):
    training_data = x_train.iloc[train_index, :]
    training_data_label = y_train.iloc[train_index]
    validation_data = x_train.iloc[val_index, :]
    validation_data_label = y_train.iloc[val_index]

In [ ]:
cv_model=model.fit(training_data, training_data_label,
			epochs=10,
            batch_size=64,
			validation_data=(validation_data, validation_data_label),
          )

Epoch 1/10
58/58 [==============================] - 1s 21ms/step - loss: 1.0946e-04 - acc: 1.0000 - val_loss: 0.0241 - val_acc: 0.9914
Epoch 2/10
58/58 [==============================] - 1s 16ms/step - loss: 9.4721e-05 - acc: 1.0000 - val_loss: 0.0241 - val_acc: 0.9924
Epoch 3/10
58/58 [==============================] - 1s 15ms/step - loss: 1.0199e-04 - acc: 1.0000 - val_loss: 0.0231 - val_acc: 0.9924
Epoch 4/10
58/58 [==============================] - 1s 16ms/step - loss: 7.7021e-05 - acc: 1.0000 - val_loss: 0.0245 - val_acc: 0.9924
Epoch 5/10
58/58 [==============================] - 1s 16ms/step - loss: 6.9710e-05 - acc: 1.0000 - val_loss: 0.0234 - val_acc: 0.9924
Epoch 6/10
58/58 [==============================] - 1s 16ms/step - loss: 6.5833e-05 - acc: 1.0000 - val_loss: 0.0232 - val_acc: 0.9924
Epoch 7/10
58/58 [==============================] - 1s 16ms/step - loss: 5.9989e-05 - acc: 1.0000 - val_loss: 0.0234 - val_acc: 0.9924
Epoch 8/10
58/58 [==============================] - 1s 

In [ ]:
results = model.evaluate(X_test, y_test, batch_size=32)

37/37 [==============================] - 0s 3ms/step - loss: 0.0292 - acc: 0.9931
